In [ ]:
import pandas as pd
import qrisklab
from qrisklab.stress_engine import StressTestEngine

pd.options.display.float_format = '{:,.4f}'.format
pd.set_option('display.max_rows', None)  # Set max_rows to None to display all rows
pd.set_option('display.max_columns', None)  # Set max_columns to None to display all columns


In [2]:
valuation_datetime = '2025-06-03 00:00:00'

In [ ]:
# Prepare market data to fit Vol Model
# dbc = qrisklab.clients.deribit.DeribitClient()
# df_real_time = dbc.fetch_deribit_option_data(currency="BTC")
df_real_time = pd.read_csv('deribit_option_data_real_time.csv')

df_real_time['instrument'] = df_real_time['instrument_name']
df_processed = qrisklab.utils.process_instruments(df_real_time,valuation_datetime[:10])
df_processed['vol'] = df_processed['mark_iv']/100
df_processed = df_processed[df_processed['put_call']=='call']
df_processed[['expiry','time_to_expiry','strike','vol','forward_price']]['expiry'].unique()

df = df_processed.copy()
df['IV'] = df['mark_iv'].astype(float)
df['Strike'] = df['strike'].astype(float)
df['Date'] = pd.to_datetime(df['expiry'])
df['Tau'] = df['time_to_expiry']
df['F'] = df['forward_price']
df['S'] = df['underlying_price']

df = df[df['Tau']>0]


In [ ]:
# Dummy Positions
df_pos = pd.DataFrame({'instrument': ['BTC-27MAR26-105000-P','BTC-5JUN25-96000-C'],'multiplier': [1,1], 'quantity': [1000,-500]})
df_pos

In [ ]:
# Stress Scenarios
SCENARIOS = [
    {'spot_shock':0.1, 'vol_shock':0},
    {'spot_shock':-0.1, 'vol_shock':0},
    {'spot_shock':0.2, 'vol_shock':0.25},
    {'spot_shock':-0.2, 'vol_shock':0.5}
]

In [ ]:
# setup stress engine
st_engine = StressTestEngine()
st_engine.set_valuation_datetime(valuation_datetime)
st_engine.update_vol_curves(df)
st_engine.update_positions(df_pos,'BTC')


In [ ]:
# run stress tests
df_re = pd.DataFrame()

for sc in SCENARIOS:
    re = st_engine.shock(**sc)
    df_re = pd.concat([df_re,re])

df_re = df_re.reset_index(drop=True)
df_re


,instrument,multiplier,quantity,underlying,expiry,strike,put_call,underlying-expiry,time_to_expiry,cost_of_carry_rate,rate,snapshot_timestamp,valuation_datetime,valuation_date,spot_shock,spot,post_shock_spot,post_shock_vol,vol,vol_shock,stress_pnl,shock_mode
0,BTC-27MAR26-105000-P,1,1000,BTC,2026-03-27,"105,000.0000",put,BTC-27MAR26,0.8137,default,0.0000,1748880000000,2025-06-03 00:00:00,2025-06-03,0.1000,"105,244.9414","115,769.4355",0.4943,0.4915,0.0000,"-3,804,378.5840",sticky_delta
1,BTC-5JUN25-96000-C,1,-500,BTC,2025-06-05,"96,000.0000",call,BTC-5JUN25,0.0055,default,0.0000,1748880000000,2025-06-03 00:00:00,2025-06-03,0.1000,"105,244.9414","115,769.4355",0.5846,0.5740,0.0000,"-5,250,697.2159",sticky_delta
2,BTC-27MAR26-105000-P,1,1000,BTC,2026-03-27,"105,000.0000",put,BTC-27MAR26,0.8137,default,0.0000,1748880000000,2025-06-03 00:00:00,2025-06-03,-0.1000,"105,244.9414","94,720.4472",0.4908,0.4915,0.0000,"4,750,080.0037",sticky_delta
3,BTC-5JUN25-96000-C,1,-500,BTC,2025-06-05,"96,000.0000",call,BTC-5JUN25,0.0055,default,0.0000,1748880000000,2025-06-03 00:00:00,2025-06-03,-0.1000,"105,244.9414","94,720.4472",0.5175,0.5740,0.0000,"4,376,742.1219",sticky_delta
4,BTC-27MAR26-105000-P,1,1000,BTC,2026-03-27,"105,000.0000",put,BTC-27MAR26,0.8137,default,0.0000,1748880000000,2025-06-03 00:00:00,2025-06-03,0.2000,"105,244.9414","126,293.9296",0.6236,0.4915,0.2500,"-2,062,983.0823",sticky_delta
5,BTC-5JUN25-96000-C,1,-500,BTC,2025-06-05,"96,000.0000",call,BTC-5JUN25,0.0055,default,0.0000,1748880000000,2025-06-03 00:00:00,2025-06-03,0.2000,"105,244.9414","126,293.9296",1.0305,0.5740,0.2500,"-10,512,944.3847",sticky_delta
6,BTC-27MAR26-105000-P,1,1000,BTC,2026-03-27,"105,000.0000",put,BTC-27MAR26,0.8137,default,0.0000,1748880000000,2025-06-03 00:00:00,2025-06-03,-0.2000,"105,244.9414","84,195.9531",0.7395,0.4915,0.5000,"18,046,851.4898",sticky_delta
7,BTC-5JUN25-96000-C,1,-500,BTC,2025-06-05,"96,000.0000",call,BTC-5JUN25,0.0055,default,0.0000,1748880000000,2025-06-03 00:00:00,2025-06-03,-0.2000,"105,244.9414","84,195.9531",1.0974,0.5740,0.5000,"4,624,120.9579",sticky_delta


In [ ]:
# Check fitted vol parameters
st_engine.svi_df

,Date,a,b,rho,m,sigma,F,S,time_to_expiry,tt
0,2025-06-04,0.0001,0.0077,-0.1862,-0.0001,0.0204,"105,234.0900","105,244.9414",0.0027,0.0027
1,2025-06-05,-0.0003,0.0179,-0.2639,-0.0119,0.0506,"105,234.0900","105,255.0624",0.0055,0.0055
2,2025-06-06,-0.0003,0.0248,-0.3646,-0.0289,0.0561,"105,234.0900","105,266.1800",0.0082,0.0082
3,2025-06-13,-0.0128,0.0641,0.0056,0.0212,0.2598,"105,234.0900","105,382.6700",0.0274,0.0274
4,2025-06-20,-3.9292,1.0000,0.3800,1.7746,4.2559,"105,234.0900","105,517.2250",0.0466,0.0466
5,2025-06-27,-0.0134,0.1021,0.1408,0.1027,0.2149,"105,234.0900","105,653.5700",0.0658,0.0658
6,2025-07-25,-0.0625,0.1673,-0.4478,-0.2817,0.6024,"105,234.0900","106,216.3600",0.1425,0.1425
7,2025-08-29,-0.0261,0.1633,-0.5325,-0.3207,0.5466,"105,234.0900","106,977.9200",0.2384,0.2384
8,2025-09-26,-0.1274,0.2228,-0.2626,-0.1824,0.8990,"105,234.0900","107,588.7500",0.3151,0.3151
9,2025-12-26,-0.0320,0.2182,-0.3493,-0.2386,0.7901,"105,234.0900","109,413.1000",0.5644,0.5644
